In [1]:
import torch
from torchvision import transforms
from PIL import Image
import os
import pandas as pd
import json

/home/aleksandra/Documents/hackathon/knum_venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def read_in_data(dir_str, ref_json):

    id_to_img = {}
    df = pd.DataFrame(columns=['id', 'image_id', 'bbox', 'category_id', 'img'])

    for line in ref_json["images"]:
        filename = line["file_name"]
        img = Image.open(dir_str + "/" + filename)
        id_to_img[line["id"]] = img

    for box in ref_json["annotations"]:
        box_coords = box["bbox"]
        # im1 = im.crop((left, top, right, bottom))
        # [lewy górny róg, szerokość, wysokość]
        box_coords = [box_coords[0], box_coords[1], box_coords[0] + box_coords[2], box_coords[1] + box_coords[3]]
        
        image = id_to_img[box["image_id"]].crop(box_coords)
        image_ = Image.new("RGB", image.size, (255, 255, 255))
        image_.paste(image, mask=image.split()[3]) 

        box["img"] = image_
        df.loc[box["id"]] = box


    df.set_index("id")
    return df

In [6]:
with open("../public_dataset/reference_images_part1.json") as f:
    ref_json = json.load(f)

dir_str = "../public_dataset/reference_images_part1"
df = read_in_data(dir_str, ref_json)
df.head()

,id,image_id,bbox,category_id,img
12,12,4,"[665, 448, 206, 174]",8,"[[[tensor(0.4353), tensor(0.4627), tensor(0.42..."
13,13,4,"[475, 186, 236, 186]",8,"[[[tensor(0.3176), tensor(0.3176), tensor(0.31..."
14,14,4,"[962, 76, 229, 217]",8,"[[[tensor(0.1686), tensor(0.1608), tensor(0.16..."
15,15,4,"[778, 250, 163, 135]",8,"[[[tensor(0.3412), tensor(0.3412), tensor(0.34..."
16,16,4,"[184, 377, 244, 194]",8,"[[[tensor(0.2314), tensor(0.2275), tensor(0.22..."


In [4]:
# h_max = 0
# w_max = 0

# for img in df.img:
#     h,w = img.size
#     h_max = max(h, h_max)
#     w_max = max(w, w_max)
    
# print(h_max, w_max)